Comparação de ensembles: Tarefa 7 da disciplina Reconhecimento de Padrões. 

Professor: Francisco de Assis Boldt 

Aluno: Diogo de Santana Candido

In [87]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.8.10


1. Use uma base de dados para classificação com pelo menos 1000 amostras;

O dataset utilizado foi o "Dados de risco a saúde materna" que pode ser obtido no link: https://archive.ics.uci.edu/ml/datasets/Maternal+Health+Risk+Data+Set#

Esses dados foram coletados de diferentes hospitais, clínicas comunitárias, cuidados de saúde materna das áreas rurais de Bangladesh por meio do sistema de monitoramento de risco baseado em IoT. 

In [88]:
import pandas as pd
df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00639/Maternal%20Health%20Risk%20Data%20Set.csv")
X, y = df.iloc[:, :6], df.iloc[:, -1]
X.shape, y.shape

((1014, 6), (1014,))

In [89]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, random_state=42)
X_tr.shape, X_te.shape, y_tr.shape, y_te.shape

((760, 6), (254, 6), (760,), (254,))

2. Escolha pelo menos três algoritmos de classificação;

In [90]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


for Algoritmo in [KNeighborsClassifier, GaussianNB, Perceptron, DecisionTreeClassifier, RandomForestClassifier]:
    modelo = Algoritmo()
    modelo.fit(X_tr, y_tr)
    y_pr = modelo.predict(X_te)
    hits = y_pr == y_te
    print(modelo, sum(hits)/len(hits))

KNeighborsClassifier() 0.6653543307086615
GaussianNB() 0.5787401574803149
Perceptron() 0.4881889763779528
DecisionTreeClassifier() 0.7913385826771654
RandomForestClassifier() 0.8031496062992126


3. Combine os classificadores de duas formas diferentes: Voting e Stacking

In [91]:
#Voting Classifier

from sklearn.ensemble import VotingClassifier
modelo = VotingClassifier([
    ('knn', KNeighborsClassifier()),
    ('naivebayes', GaussianNB()),
    ('perceptron', Perceptron()),
    ('arvore', DecisionTreeClassifier()),
    ('randomforest', RandomForestClassifier(random_state=42))
])
modelo.fit(X_tr, y_tr)
y_pr = modelo.predict(X_te)
hits = y_pr == y_te
sum(hits)/len(hits)

0.7992125984251969

In [92]:
# Stacking Classifier

from sklearn.ensemble import StackingClassifier

modelo = StackingClassifier([
    ('knn', KNeighborsClassifier()),
    ('naivebayes', GaussianNB()),
    ('perceptron', Perceptron()),
    ('arvore', DecisionTreeClassifier()),
    ('randomforest', RandomForestClassifier(random_state=42))
])
modelo.fit(X_tr, y_tr)
y_pr = modelo.predict(X_te)
hits = y_pr == y_te
sum(hits)/len(hits)


0.7677165354330708

In [93]:
# Voting e Stacking Classifier
from sklearn.ensemble import ExtraTreesClassifier

voting = VotingClassifier([
    ('knn', KNeighborsClassifier()),
    ('naivebayes', GaussianNB()),
    ('perceptron', Perceptron()),
    ('arvore', DecisionTreeClassifier()),
    ('randomforest', RandomForestClassifier(random_state=42))
])

modelo = StackingClassifier([
    ('voting', voting),
    ('extratrees', ExtraTreesClassifier(random_state=42))

])
modelo.fit(X_tr, y_tr)
y_pr = modelo.predict(X_te)
hits = y_pr == y_te
sum(hits)/len(hits)


0.7913385826771654

4. Use gridsearch (ou randomsearch) para ajustar tantos os classificadores fracos quanto o ensemble;

In [94]:
from sklearn.model_selection import GridSearchCV
import numpy as np

par_knn = {'n_neighbors': [3,5,7]}
gs_knn = GridSearchCV(KNeighborsClassifier(), par_knn)

par_nb = {'var_smoothing': np.logspace(0,-9, num=100)}
gs_naivebayes = GridSearchCV(GaussianNB(), par_nb)

par_per = {'alpha': [0.0001, 0.0003, 0.001, 0.003, 0.01, 0.03, 0.1, 0.3]}
gs_perceptron = GridSearchCV(Perceptron(), par_per)

par_arvore = {'max_depth': [10, 15, 25]}
gs_arvore = GridSearchCV(DecisionTreeClassifier(), par_arvore)

par_randomforest = {'max_depth': [10, 15, 25], 'n_estimators': [100, 150, 200]}
gs_randomforest = GridSearchCV(RandomForestClassifier(random_state=42), par_randomforest)

voting = VotingClassifier([
    ('gs_knn', gs_knn),
    ('gs_naivebayes', gs_naivebayes),
    ('gs_perceptron', gs_perceptron),
    ('gs_arvore', gs_arvore),
    ('gs_randomforest', gs_randomforest)
])

modelo = StackingClassifier([
    ('voting', voting),
    ('extratrees', ExtraTreesClassifier(random_state=42))
])
modelo.fit(X_tr, y_tr)
y_pr = modelo.predict(X_te)
hits = y_pr == y_te
sum(hits)/len(hits)



0.7952755905511811

5. Coloque o Jupyter Notebook no seu GitHub.